## Useful Advice

- How can we update/store the model for updating when new users/new items/ new interactions come in?
    - The most robust answer is that you periodically recompute the model from scratch to handle all three. For just new interactions, you can run additional fitting iterations on the same model with the new data. (You can persist the model by pickling it.) Adding new users/items (this is called fold-in) is somewhat tricky, and not explicitly supported. For models that naturally take in new information (and new users), you should have a look at [sequence-based models](https://maciejkula.github.io/spotlight/index.html#sequential-models)
- Getting similar items
    - https://github.com/lyst/lightfm/issues/244
- bdicts library to convert UUIDs to 32 bit ints for recommender libraries
    - https://stackoverflow.com/questions/48068147/recommender-systems-convert-uuids-to-32-bit-ints-for-recommender-libraries
- Evaluation:
    - The scores themselves have no meaning in isolation; they are only meaningful because they define a ranking over items for a given user. The scale they take depends on the loss you specify, the learning rate, the regularization parameters, and the data itself. I recommend keeping an eye on the MRR/AUC scores of your model, and comparing them with what a random or a popularity model would achieve.
- Mini-Batch Training for large dataset:
    - https://github.com/lyst/lightfm/issues/234
- Item-Item Recommendation ???
    - https://github.com/lyst/lightfm/issues/239#issuecomment-352774493
- Sample weight for different interactions
    - https://github.com/lyst/lightfm/issues/260
- Handling non-categorical features
    - https://github.com/lyst/lightfm/issues/261
- Explaining Recommendation
    - try using item similarity to do that present support for your recommendations. For example, you could use cosine similarity between the embedding of the item you are recommending and items that the user has interacted with in the past to find items from the user's history that are most similar to the recommendation you are making.
    - https://github.com/lyst/lightfm/issues/251#issuecomment-363314396
- One interesting characteristic of WARP is that it is effectively playing against itself: it is preferentially using examples that it scores highly as negative examples. Up to a certain point, this allows it to express user preferences better; beyond that point, it is self-defeating, as it is selecting the examples that it should recommend and treating them as negatives. The extent to which that happens is governed by the max_sampled hyperparameter. I suspect that this is what happening in your case. Instead of switching to BPR, can you try decreasing that hyperparameter? Otherwise, I don't see any obvious problems with switching losses. It's interesting that you find different results with adagrad and adadelta, I haven't observed that.
    - https://github.com/lyst/lightfm/issues/258#issuecomment-363313831
- Use sample weight to incorporate multiple feedbacks
    - https://github.com/lyst/lightfm/issues/258#issuecomment-363313831
- Contextual-Awareness: If you want to have contextual features (time-varying, or specific to a given interaction instance), you could integrate them in either user or item features. This way you would have more than one features row for a given user or item. For example, instead of having one row for item 10: {'id': 10, 'category': 'foo'} you could have two rows: {'id': 10, 'category': 'foo', 'time-of-week': 'friday'} and {'id': 10, 'category': 'foo', 'time-of-week': 'tuesday'}. With a little ingenuity in feature engineering you can make the model very flexible.
    - https://github.com/lyst/lightfm/issues/238#issuecomment-352774054
- To incorporate numerical side features, we may find that discretization works better.
    - https://github.com/lyst/lightfm/issues/261
- Using predict with feature matrix
    - https://github.com/lyst/lightfm/issues/240
- Get item recommendation for a particular user on a particular item page
    - https://github.com/lyst/lightfm/issues/266#issuecomment-378243505